![Steam_Logo](Steam_Logo.jpg)

**Steam** es una plataforma de distribución digital de videojuegos desarrollada por Valve Corporation. Fue lanzada en septiembre de 2003 como una forma para Valve de proporcionar actualizaciones automáticas a sus juegos, pero finalmente se expandió para incluir juegos de terceros.

# Paso 1: Comenzando nuestro proceso ETL
Comenzamos leyendo el Glosario de Datos para comprender la estructura y el contenido de los tres archivos proporcionados en la documentación del proyecto:

1. **steam_games.json.gz**: Este conjunto de datos contiene información completa sobre los juegos disponibles en Steam. Incluye detalles como títulos de juegos, géneros, fechas de lanzamiento, etc., lo que nos permite obtener información sobre el **catálogo de juegos**.

2. **user_reviews.json**: El conjunto de datos de reseñas de usuarios proporciona comentarios generados por los usuarios para los juegos de Steam. Contiene datos como el texto de la reseña y los votos de retroalimentación de calificación, lo que nos permite analizar los sentimientos y experiencias de los usuarios de manera efectiva.

3. **users_items.json**: En este conjunto de datos, encontramos información relacionada con los usuarios de Steam y sus colecciones de objetos dentro del juego. Los datos interesantes de este conjunto incluyen las preferencias de los usuarios y los objetos específicos que poseen.

Una vez que revisamos la estructura, las definiciones de columnas y la configuración, procedimos a cargarlos en Dataframes de Pandas para acceder a la información y transformarla y procesarla.<br>
<br>

## 1.1 Juegos

Procedimos con el [ETL_juegos](ETL_juegos.ipynb): **steam_games.json.gz** sin inconvenientes:<br>

El dataframe se llamó df_juegos y contiene:<br>
* 120.445 filas
* 13 columnas
<br>
Al usar Data Wrangler, obtenemos una idea general del conjunto de datos. Contiene muchos valores faltantes; la mayoría de las columnas tienen hasta un 70% de valores faltantes en el conjunto de datos.<br>

> Eliminaremos la información faltante, ya que, en primer lugar, no podemos completar la información faltante debido a su diversidad y origen desconocido.<br>
> Además, eliminar la información faltante aún nos proporciona suficiente información para nuestro proyecto. No hay pérdida.

Ahora nos quedan 22.530 filas y 13 columnas de información.<br>
Ahora debemos evaluar cada columna para decidir cuáles son importantes para el proyecto y cuáles se pueden eliminar.<br>
<br>
Para continuar, eliminaremos los duplicados: solo se eliminaron 2 registros.

#### Ahora podemos comenzar a definir qué columnas debemos conservar. <br>
Las columnas que ya sabemos que son necesarias para las funciones de la API son: <br>

*   **developr** para la función **def desarrollador**
*   **genre** para la función **def usuario_por_género**
*   **release_date** para la función **def mejor_desarrollador_año**
*   **price** para la función **def datos_usuario**
*   **title** para **def mejor_desarrollador_año**
*   **id** identifica cada juego por su valor único de id
*   **publisher** para la función **def desarrollador**

##### Estas columnas anteriores se transformarán y manipularán como parte del modelo.


#### También podemos comenzar a evaluar la información de otras columnas para eliminar las que no necesitamos de nuestro conjunto de datos.<br>
##### Estas son:
*   **app_name** información similar a la del título
*   **url** no aporta valor al proyecto previsto
*   **tags** similar a los géneros y no aporta valor al proyecto previsto
*   **discount** no aporta valor al proyecto previsto
*   **reviews_url** no aporta valor al proyecto previsto
*   **specs** no aporta valor al proyecto previsto
*   **early_access** no aporta valor al proyecto previsto

##### Por lo tanto, todas las columnas anteriores se eliminarán.

Añadiendo valor a nuestros datos:
*   Convertimos **price** en una columna completamente de tipo float, de modo que toda la información sobre gratuitos y promociones se convierte en ceros.
*   Creamos una nueva columna = **release_year** y eliminamos la columna anterior **release_date**. Esta nueva columna se utilizará en la función **top_developer**.
*   Eliminamos todas las columnas que no aportan al modelo o las funciones.

#### Ahora queda una última columna que debemos analizar y transformar: 'géneros'
##### géneros
Esta columna contiene una lista en cada una de las filas que indica a qué géneros pertenece el juego. <br>
Un **genre** es una categoría que agrupa juegos según sus objetivos, historia y jugabilidad.<br>
Ayuda a los usuarios y desarrolladores a identificar juegos con estilos y temas similares, como los desafíos intensos en los juegos de acción o el desarrollo de personajes y las misiones en los juegos de rol.<br>
En esta columna, para cada juego (fila), encontramos una lista de géneros que presenta el juego. Esta información se utilizará en la función **user_for_genre** y en nuestro modelo de recomendación. Sin embargo, abordaremos esto en nuestro análisis exploratorio de datos (EDA); por ahora, continuaremos con nuestras etapas de ETL.

Hemos almacenado el dataframe transformado en [games.parquet](games.parquet) en la carpeta "data".

## 1.2 Reseñas de Usuarios

Al pasar al segundo archivo [ETL_user_reviews](ETL_user_reviews.ipynb), nos enfrentamos a nuestro primer desafío:<br>
El procedimiento estándar para leer el archivo utilizado en el primer archivo resultó en errores tras errores, porque los datos dentro del archivo .json no estaban en el formato esperado.<br>
Después de investigar, aprendí que cuando el archivo JSON que intentas cargar no está correctamente formateado (como es el caso aquí), puedes evaluarlo línea por línea utilizando **ast.literal_eval()**. <br>

Después de buscar extensamente y profundizar en Stack Overflow, encontré un artículo que fue un avance:

[Convert JSON to pd.DataFrame](https://stackoverflow.com/questions/55338899/convert-json-to-pd-dataframe/65427497#65427497)<br>
[JSONDecodeError](https://bobbyhadz.com/blog/python-jsondecodeerror-expecting-property-name-enclosed-in-double-quotes)<br>

El dataframe **df_reviews** (**user_reviews.json.gz**) contiene:<br>
*   25.485 filas
*   3 columnas
<br>
Primero procedemos a identificar valores faltantes o duplicados.
Encontramos 623 registros duplicados, validamos que realmente estén duplicados y finalmente eliminamos el valor duplicado, conservando solo la primera ocurrencia del valor.<br>
Luego avanzamos a la columna de reseñas, que es de gran interés para nuestro proyecto.<br>

Fue fácil detectar que la tercera columna es una colección de listas de diccionarios, y que los datos dentro de la última columna son vitales para el desarrollo del análisis de sentimientos.

```
df_reviews.iloc[100,2]
```
[{'funny': '',
  'posted': 'Posted October 13, 2014.',
  'last_edited': '',
  'item_id': '209870',
  'helpful': '3 of 8 people (38%) found this review helpful',
  'recommend': True,
  'review': 'Its a very fun game i recomend as its nearly like TITANFALL but its FREE!Play this game now'}]

#### Necesitamos extraer en columnas la información presente dentro de cada campo de la tercera columna. <br>
Los campos que se convertirán en columnas son: graciosos, publicados, editados por última vez, id del ítem, útil, recomendado y reseña.
Después de varios intentos con diferentes enfoques, encontré una página con una explicación de un ejercicio similar que adapté para resolver la lista anidada de diccionarios en columnas y combinaciones de registros. <br>

[Convert list of nested dictionary into Pandas dataframe](https://www.geeksforgeeks.org/python-convert-list-of-nested-dictionary-into-pandas-dataframe/)

Finalmente, el dataframe desanidado contiene:

* 59.305 filas
* 9 columnas ([‘user_id’, ‘user_url’, ‘funny’, ‘posted’, ‘last_edited’, ‘item_id’, ‘helpful’, ‘recommend’, ‘review_text’]) Por lo tanto, el resultado final es un dataframe con 25.485 filas, sin valores faltantes y las siguientes columnas:<br>
* **user_id**: identificador único para cada usuario en la plataforma.<br>
* **user_url**: URL que aloja el perfil del usuario en Steam Community.<br>
* **reviews**: contiene una lista de diccionarios.Para cada usuario, enumera todas las reseñas publicadas por el usuario:
    * **funny**: indica si otro usuario califica como graciosa la reseña del usuario
    * **posted**: fecha en que se realizó la publicación, formato: “Publicado el 21 de abril de 2011”.
    * **last_edited**: fecha en que se editó la publicación.  
    * **item_id**:es el ID del juego, único para cada juego.
    * **helpful**: otros usuarios califican si la reseña fue útil
    * **recommend**: un valor booleano que indica si el usuario recomienda los juegos o no
    * **review**: es la reseña escrita por el usuario que se publica en la URL del juego.

De todas estas columnas, encontramos dos con valores faltantes en su mayoría, y los valores faltantes en el formato ‘’ se cambiaron a None. <br> Después de revisar la cantidad de valores None en todas las columnas agregadas, decidimos eliminar las columnas:<br>

- **funny** con más del 86% de valores None.
- **last_edited** con más del 89% de valores None.

Luego procedimos a transformar la columna ‘publicado’ de un formato ‘Publicado {mes} {día}, {año}’ en una nueva columna ‘fecha’. En esta columna encontramos 9929 registros con el año faltante, que se completaron con el año actual (2024) para diferenciarlos del resto de los valores extraídos correctamente. Estos registros de 2024 no se considerarán para la función mejor_desarrollador_año, pero la información restante aún está disponible para el modelo.<br>

El dataframe resultante [reviews.parquet](data\reviews.parquet) resume 58.400 filas y 9 columnas.<br>

## 1.2 User_Items
Finalmente, avanzamos con nuestro tercer y último archivo [ETL_reviews](01.%20ETL/ETL_reviews.ipynb): **users_items.json.gz**
Obtenemos el mismo error que en el segundo archivo, por lo que debemos cargar la información utilizando la función literal de ast.<br>

Una vez más, encontramos otra columna, items, con una lista anidada de diccionarios, y la información en esta columna es la lista de elementos (juegos) que cada usuario compró. Esta es toda la información valiosa que necesitamos para desarrollar nuestras funciones para nuestro modelo.<br> 
Antes de desanidar la columna ‘items’, verificamos una vez más los duplicados utilizando el campo ‘user_id’.<br> 
Encontramos 1357 registros duplicados, que después de validar que estaban duplicados, procedimos a eliminarlos y mantener solo la primera ocurrencia.<br>

Finalmente, procedemos como con el archivo de reseñas anterior, aprendiendo cuál es la estructura de los datos que necesitamos extraer de la columna ‘items’.<br>

Las columnas que agregaremos a nuestro df_items son:

```
df_items.iloc[10,4]
```
[{'item_id': '4000',
  'item_name': "Garry's Mod",
  'playtime_forever': 2644,
  'playtime_2weeks': 0},
 {'item_id': '1250',
  'item_name': 'Killing Floor',
  'playtime_forever': 30266,
  'playtime_2weeks': 0},
 {'item_id': '35420',
  'item_name': 'Killing Floor Mod: Defence Alliance 2',
  'playtime_forever': 54,
  'playtime_2weeks': 0},

...
  'playtime_2weeks': 462},
 {'item_id': '444640',
  'item_name': 'Bloons TD Battles',
  'playtime_forever': 809,
  'playtime_2weeks': 0}]

  Entonces necesitamos iterar a través del campo de elementos para cada fila, con el fin de extraer para cada usuario la siguiente información:<br>
  - **'item_id'**: '444640' --> es el identificador único del game **game_id**<br>
  - **'item_name'**: 'Bloons TD Battles',--> **item_name** nombre del game<br>
  - **'playtime_forever'**: 809,--> La cantidad total de tiempo en minutos que el usuario jugó el juego en total **minutes**<br>
  - **'playtime_2weeks'**: 0}] --> La cantidad total de tiempo en minutos que el usuario jugó el juego en **las últimas dos semanas**<br>

Entonces ejecutamos nuestro código para convertir esta información en columnas para cada usuario.<br> 
<br> 
Luego obtenemos un dataframe de:<br>

* 5.153.209 filas<br>
*8 columnas<br> <br> 
Ahora procedemos a obtener más información sobre las diferentes columnas y los datos para decidir qué columna es relevante para nuestro proyecto y cuál se puede eliminar.<br>
###  Información valiosa y datos desechables <br>
Ahora que tenemos una idea de la estructura del dataframe y la información, podemos concluir que hay muchas columnas que podemos eliminar para mantener nuestros datos manejables y lo más compactos posible.<br>

**Columnas a eliminar:**<br>
-   **items_count** es una columna que, dada la información, se puede calcular a demanda.<br>
-   **steam_id** contiene la misma información que  **user_id**
-   **user_url** es el url del usuario, y no lo utilizaremos en particular en nuestro análisis<br>
<br>

El dataframe resultante [items.parquet](data\items.parquet) resume 5.094.082 filas y 5 columnas.


# Paso 2: Resumen del Análisis EDA
Ahora que tenemos los tres archivos limpios y listos, podemos comenzar a obtener información sobre los datos que se utilizarán en nuestro proyecto de modelo de recomendación.<br>
Aquí encontrarás un resumen del trabajo realizado en el análisis exploratorio de datos (EDA). <br> 
Para obtener información más detallada y visualizaciones:<br>

[EDA_games](0.%20EDA/EDA_games.ipynb)<br>

[EDA_reviews](02.%20EDA/EDA_reviews.ipynb)<br>

[EDA_items](02.%20EDA/EDA_items.ipynb)<br>

## 2.1 Juegos
En el Análisis EDA, nos centraremos principalmente en:<br>

- buscar valores atípicos
- estadísticas y descripción de los datos
- transformación de datos
- Variables:
    - **item_id**: esta columna contiene el identificador único del juego.
    - **item_name**: este es el nombre del juego.
    - **developer**: es un desarrollador de software especializado en el desarrollo de videojuegos, el proceso y las disciplinas relacionadas con la creación de videojuegos.
    - **publisher**: una editorial de videojuegos es una empresa que publica videojuegos que han sido desarrollados internamente por la editorial o externamente por un desarrollador de videojuegos.
    - **release_year**: es el año en que se lanzó el juego.
    - **genres**: esta columna contiene una lista de géneros para cada juego, que transformaremos con diferentes enfoques.<br>

### Un resumen de lo que encontrarás en el cuaderno:
#### Precio
*   Descubrir si nuestra información de precios tiene valores atípicos.
*   Juegos más caros.
*   Precio promedio de los juegos.
*   Rango de precios.
*   Distribución de precios.
*   Estadísticas de precios.

#### Release Year
*   Rango de años.
*   Juegos lanzados por año.
*   Los cinco años con más juegos lanzados.

#### Publishers
*   Las diez editoriales con la mayor cantidad de juegos publicados.

#### Developers
*   Los diez desarrolladores con la mayor cantidad de juegos publicados.
*   Los desarrolladores menos populares o relevantes, con la menor cantidad de juegos publicados.
*   Las editoriales menos importantes, con la menor cantidad de juegos publicados.

#### Genres
*   Los géneros más populares, con la mayor cantidad de juegos publicados.
*   Los géneros menos populares, con la menor cantidad de juegos publicados.

## Análisis de correlación entre géneros y precios
### ¿Tienen los géneros un impacto en el precio de los juegos?

## 2.2 Reviews

**El propósito final de este análisis es definir, para el modelo de recomendación solicitado y las diferentes funciones, para cada conjunto de datos y función, las columnas y datos correspondientes que serán necesarios**.

### Durante el proceso ETL, ya nos ocupamos de:
[ETL_user_reviews](01.%ETL\ETL_user_reviews.ipynb)<br>

- valores faltantes y limpieza de datos
- eliminación de columnas de información irrelevante
- eliminación de filas con información inexistente o no válida.
- eliminación de valores duplicados
- corrección de todos los tipos de datos para cada dato y columna

### Análisis de sentimientos
El conjunto de datos que utilizaremos ya ha pasado por la transformación de análisis de sentimientos, por lo que, en lugar de trabajar en el análisis exploratorio de datos (EDA) con las reseñas, tendremos una columna llamada ‘sentimiento’ que muestra el resultado del proceso de análisis de sentimientos.<br>

### Por lo tanto, en el Análisis EDA, nos centraremos principalmente en:
- buscar valores atípicos
- estadísticas y descripción de los datos
- transformación de datos
- Variables:
- user_id: esta columna contiene el identificador único para cada uno de los usuarios en Steam.
- user_url: esta columna contiene la URL al perfil de cada usuario en Steam.
- item_id: esta columna contiene el identificador único para el juego.
- helpful: en esta columna, otros usuarios califican si la reseña fue útil.
- recommend: esta columna es un valor booleano que indica si el usuario - recomienda los juegos o no.
- date: es la fecha en que se publicó la reseña. A partir de esta columna, extraemos la columna year.
- year: es el año en que se lanzó el juego.
- sentiment: esta columna contiene el valor de clasificación del análisis de sentimientos:<br>
0 --> Negativo <br> 
1 --> Neutral <br> 
2  --> Positivo <br>

## Un resumen de lo que encontrarás en el notebook:
#### user_id
- Número total de usuarios en la plataforma.
- Cálculo del número promedio de reseñas por usuario.
- Exploración de la distribución del número de reseñas por usuario.
- Determinación del usuario con el mayor número de reseñas.
- Identificación del usuario con el mayor número promedio de reseñas por año.

**Hallazgos:**

* Solo el 30% de los usuarios en la plataforma posteó reseñas, esto es 25.446 unique user_id
* Este 30% de los usuarios en promedio publica 2.3 reseñas:
* De este 30%, el 53% de los usuarios publica solo una reseña
* Y menos del 2% de los usuarios que publican, publican más de 8 reseñas

#### item_id
- Cuántos juegos tienen reseñas
- Qué porcentaje de juegos tiene reseñas del total de juegos disponibles en la plataforma
- Exploración de la distribución del número de reseñas por juego
- Promedio de reseñas por juego
- Identificación de los 10 juegos con mayor número de reseñas
- Juegos con más de 500 reseñas

**Conclusiones:**

* El 16% de todos los juegos registrados en la plataforma reciben reseñas, 3,682 de un total de 22,530.
* El 15% de los juegos reciben entre 1 y 12 reseñas
* El promedio de reseñas por juego es de 15 reseñas
* El 67% de todas las reseñas se concentran en solo 12 juegos.

### Helpful
En esta columna, otros usuarios califican si la reseña fue útil.
Así es como muchos usuarios han calificado la reseña publicada como útil.
Vamos a transformar esta columna para obtener una idea de cómo otros usuarios califican la reseña del usuario.

**Conclusiones:**
- La mayoría de las reseñas no son calificadas por otros usuarios, el 61% de ellas no son calificadas.
- El 17% de las reseñas calificadas son consideradas útiles por otros usuarios.

## Análisis de Recomendación y Sentimiento
**Realizamos un análisis bivariado**<br>

En el caso de estas dos variables, vamos a realizar el EDA como un análisis bivariable.
Esta columna es un valor booleano que indica si el usuario recomienda los juegos o no. Esto es, si la recomendación es positiva, hay un 1 y si no, un cero.
Podemos usar esta columna para evaluar o validar nuestro resultado del análisis de sentimiento.
- Cuántos usuarios publican una recomendación positiva
- Cuántos usuarios publican una recomendación positiva y una reseña de sentimiento positivo
- Cuántas recomendaciones positivas coinciden con un sentimiento positivo
- Cuántos sentimientos negativos coinciden con una recomendación negativa

#### Análisis de Sentimiento y Recomendación
Vamos a considerar que todos los sentimientos neutros son positivos, y vamos a realizar un tipo de puntuación de precisión al análisis de sentimiento utilizando las bibliotecas NLTK y TextBlob dada la columna de recomendación.
Esto es para analizar de alguna manera qué tan precisa es nuestra análisis de sentimiento.

**Conclusiones**
A partir de la evaluación de precisión entre ambos análisis de sentimiento, concluimos que el análisis NLTK es el que tiene mejor rendimiento.
Ahora podemos revisar los resultados en comparación con la recomendación del usuario:

* **Negativos** Podemos ver que para la recomendación Verdadera, es decir, que el usuario recomienda el juego incluso si la reseña es negativa, el porcentaje de recomendación es de aproximadamente el 70%, lo que significa que la mayoría de los usuarios, aunque consideren que el juego no es bueno, todavía lo recomiendan.
* **Neutral** En este caso, la reseña siendo neutral no es una reseña negativa, por lo que no es sorprendente que los usuarios aún recomienden el juego incluso si la reseña no es mala. Y además, nuestro análisis de sentimiento funciona muy bien reconociendo reseñas positivas pero no tan bien clasificando reseñas neutrales.
* **Positivos** En este caso, confirmamos lo que nuestro test de precisión mostró, nuestro análisis de sentimiento es bueno clasificando reseñas positivas, y las reseñas positivas coinciden en el 91% de los casos.
Dado que nuestro análisis fue un enfoque rápido y simple con muchas oportunidades, los resultados son buenos, considerando todo.

## 2.3 Items
### Durante el proceso ETL ya nos ocupamos de:

[ETL_items](01.%20ETL/ETL_items.ipynb)<br>

-   valores faltantes y limpieza de datos
-   eliminación de columnas de información irrelevante
-   eliminación de filas con información no existente o no válida
-   eliminación de valores duplicados
-   corrección de todos los tipos de datos para cada columna y dato

### Entonces en el **Análisis EDA** nos enfocaremos principalmente en:
-   búsqueda de valores atípicos
-   estadísticas y descripción de datos
-   transformación de datos
-   Variables:
    - user_id: Este user_id es el identificador único para cada uno de los usuarios en la columna de la plataforma
    - item_id: identificador único del juego en la plataforma
    - item_name: nombre del juego o aplicación
    - playtime_forever: El número total de minutos jugados "registrados"
    - playtime_2weeks: El número total de minutos jugados en las últimas 2 semanas
-   Análisis multivariable:

### Un resumen de lo que encontrarás en el cuaderno:

#### User_id
Este user_id es el identificador único para cada uno de los usuarios en la plataforma.
**Hallazgos**
- Cuántos usuarios están registrados en la plataforma

#### item_id
- Cuántos juegos son elementos en las cuentas de los usuarios
- Porcentaje del total de juegos
- Promedio de cantidad de juegos por usuario
- Los 10 elementos más vendidos (más usuarios lo tienen)
- Los 10 juegos más jugados en las últimas 2 semanas

#### Atípicos para item_id por usuario

**Hallazgos**
* La cantidad promedio de juegos por usuario es de 71 juegos
* Más del 50% de los usuarios poseen más de 40 juegos cada uno.
* Solo hay 39 usuarios con más de 2000 juegos cada uno, esto se considerará como atípicos pero representan menos del 0.056%
* La **cantidad máxima** de juegos propiedad de un usuario es de *7762 juegos*
- Los 10 elementos más vendidos (más usuarios lo tienen)

#### item_name
Este es el nombre del juego.

### playtime_forever
El número total de minutos jugados "registrados"
- 	Top 5 usuarios con más minutos jugados en total
-	Top 5 juegos con más minutos jugados
-	Promedio de minutos jugados por usuario
-	Promedio de minutos por juego

**Hallazgos**
* Los 10 mejores juegos representan aproximadamente el 41% del tiempo total de juego.
* El tiempo de juego promedio por usuario es de 1497 minutos
* El 50% de todos los usuarios juegan más de 887 minutos.
* El promedio de minutos por juego es de 277 minutos
* El 50% de los juegos se juegan durante más de 90 minutos en promedio

### playtime_2weeks
El número total de minutos jugados "registrados" en las últimas 2 semanas
- 	Top 5 usuarios con más minutos jugados en total
-	Top 5 juegos con más minutos jugados
-	Promedio de minutos jugados por usuario
-	Promedio de minutos por juego

# 3. Feature Engineering

## 3.1 Análisis de Sentimiento
Para obtener una idea y un valor real de las opiniones de nuestros usuarios, realizaremos un Análisis de Sentimiento a partir de las revisiones de nuestros usuarios.
Este **Análisis de Sentimiento** luego se utilizará en nuestro sistema de recomendación, modelo.<br>
Para esto, elegimos trabajar con las bibliotecas de Python NLTK.<br>
No hay un enfoque único para el Análisis de Sentimiento, todos los métodos pueden ofrecer un buen resultado.
En nuestro proyecto elegimos la biblioteca NLTK y no TextBlob, que es un enfoque mucho más simple:

NLTK (Natural Language Toolkit):

**Ventajas**:
- Proporciona una amplia gama de herramientas y recursos para tareas de procesamiento del lenguaje natural.
- Flexible y personalizable para tareas avanzadas de PNL.
- Comunidad bien establecida y extensa documentación.

**Desventajas**:
- Requiere más código y esfuerzo para tareas simples en comparación con TextBlob.
- Puede ser más lento para ciertas tareas debido a su naturaleza completa.

En nuestro caso, estudiamos e implementamos ambos métodos, puedes revisar en detalle en [Feature Engineering](03.%20Feature%20Engineering.ipynb).

Procedimos siguiendo el proceso de:

1. **Preprocesamiento de Datos y Tokenización**: Este paso implica limpiar y preparar los datos de texto para el análisis. Incluye tareas como eliminar puntuación, convertir texto a minúsculas, eliminar palabras vacías y manejar caracteres especiales. La tokenización implica descomponer el texto en palabras o tokens individuales.
> Elegimos realizar esto creando una nueva columna, llamada 'clean_review' y pasando una función al contenido de la columna 'review_text' y almacenando esta transformación, para cada elemento en 'clean_review'.

2. **Análisis de Sentimiento**: Una vez que los datos de texto están preparados, se puede realizar el análisis de sentimiento. Hay varias bibliotecas y técnicas disponibles para el análisis de sentimiento, incluidos enfoques basados en léxicos, modelos de aprendizaje automático y modelos de aprendizaje profundo. Establecimos definir la clasificación del sentimiento en:
* Negativo = 0 para todos los valores donde el compound_score fue menor a -0.05
* Neutral = 1 todos los otros resultados ni negativos ni positivos y ninguno valores o emojis.
* Positivo = 2 para todos los valores donde el compound_score fue mayor a 0.05
> Para nuestro trabajo, elegimos trabajar con Vader. Aunque Textblob es una solución adecuada porque es mucho más rápida y simple, al comparar ambos resultados, la única desventaja de NLTK fue el uso de recursos, el modelo se ejecuta lentamente, pero el resultado es mejor.
>NLTK es más asertivo en la clasificación de sentimientos negativos y positivos, pero aún mejor para la clasificación neutral.
En el [Feature Engineering](03.%20Feature%20Engineering.ipynb) puedes revisar en detalle el proceso para ambos enfoques y nuestro análisis, y luego en el [EDA_reviews](EDA_reviews.ipynb) puedes encontrar la comparación de los resultados para ambos enfoques y más exploración.<br>

3. **Análisis, Visualización y Resumen**: Finalmente, los resultados del análisis de sentimiento pueden analizarse y visualizarse para obtener información de los datos.

* Recursos:<br>
[Análisis de Sentimiento](https://www.youtube.com/watch?v=O_B7XLfx0ic)

[Análisis de Sentimiento NLTK](https://www.youtube.com/watch?v=XFoehWRzG-I)

[Artículo de Análisis de Sentimiento](https://www.kdnuggets.com/sentiment-analysis-in-python-going-beyond-bag-of-words)

## 3.2 Calificación de las revisiones de nuestros usuarios en el dataframe de revisiones.
## Transformación de Revisiones
### Necesitamos transformar las revisiones de nuestros usuarios, el resultado del botón de recomendación y la calificación de útil (donde otros usuarios calificaron la revisión) en una calificación para alimentar nuestro sistema de recomendación.<br>
Para esto creamos una función que considerará los tres elementos, cuando existan y promediará una calificación que exprese la revisión del usuario en números.<br>
En general, esta función asigna una puntuación de calificación en función de los valores proporcionados para 'rate', 'sentiment' y 'recommend', considerando diferentes combinaciones de estos valores y siguiendo reglas específicas.<br>
*   Si la columna 'rate' es mayor que 0:
        -Si la columna 'recommend' es 0:
            -Si la columna 'sentiment' es 0, la puntuación de calificación se establece en 0.
            -Si la columna 'sentiment' es 1, la puntuación de calificación se calcula como 50 veces el 'rate'.
            -Si la columna 'sentiment' es 2, la puntuación de calificación se calcula como 75 veces el 'rate'.
        -Si la columna 'recommend' es 1:
            -Si la columna 'sentiment' es 0, la puntuación de calificación se calcula como 50 veces el 'rate'.
            -Si la columna 'sentiment' es 1, la puntuación de calificación se calcula como 75 veces el 'rate'.
            -Si la columna 'sentiment' es 2, la puntuación de calificación se calcula como 100 veces el 'rate'.
*   Si la columna 'rate' es 0:
    - Si la columna 'recommend' es 0:
        - Si la columna 'sentiment' es 0, la puntuación de calificación se establece en 0.
        - Si la columna 'sentiment' es 1, la puntuación de calificación se establece en 50.
        - Si la columna 'sentiment' es 2, la puntuación de calificación se establece en 75.
    - Si la columna 'recommend' es 1:
        - Si la columna 'sentiment' es 0, la puntuación de calificación se establece en 50.
        - Si la columna 'sentiment' es 1, la puntuación de calificación se establece en 75.
        - Si la columna 'sentiment' es 2, la puntuación de calificación se establece en 100
Luego normalizamos esta calificación en una puntuación de 1 a 5.<br>

Esta calificación será el motor de nuestro Sistema de Recomendación de Filtrado Colaborativo y el Recomendador KNN de Aprendizaje Automático <br>

## 3.3 Funciones de API 
En esta etapa del proyecto, los dataframes trabajados vamos a experimentar y trabajar en crear nuestras Funciones API solicitadas.<br>
Estas son :<br>
1. **Developer** número de juegos y porcentaje de contenido gratuito por desarrollador por año.<br>
2. **User_data**: cuánto dinero ha gastado el usuario, qué porcentaje, del total de juegos que posee el usuario, ha recomendado el usuario de las revisiones.recomendar y cuántos juegos ha comprado.<br>
3. **User_for_genres**: esta función debe devolver el usuario con la mayor cantidad de minutos para los géneros dados y una lista de minutos acumulados por año desde la fecha de lanzamiento<br>
4. **Best_developer_year**: esta función devuelve los 3 mejores desarrolladores basados en la mayor cantidad de recomendaciones para el año dado (revisiones.recomendar = True y sentiment = 2)<br>
5. **Developer_Reviews_Analyis**: dado un desarrollador, la función devuelve un diccionario con el desarrollador como clave y la cantidad de Reseñas de Sentimiento Positivo y Negativo.<br> 


## 3.4 Funciones y dataframes para el deploy en Render <br>
Luego de realizar todo el proceso de creación de las funciones no tuve en cuenta el peso de los dataframes que resultaron. Cuando hice el testing en local con FASTAPI, las funciones y todo el proceso corría sin errores ni problemas. 
Al haber guardado todo en formato .parquet el repositorio completo no superaba los 80MB.<br>
Pero al realizar el deploy en render, no corria por que se excedian los 500MB permitidos en modo Free de Render.<br>
Como resultado, lo que se trabajó en especial fue con el df_items que contaba con mas de 5MM de registros y que en .parquet pesaba solo 28MB, pero que al procesarlo con la funcion en render excedía los 512MB disponibles.<br>
Para ello, lo que decidí hacer, para poder cumplir con el deploy, fue filtrar el dataframe items para reducirlo:<br>
Aca debajo esta el detalle del trabajo y lo decidido:<br>
 
>Agrupamos por user_id y contamos la cantidad de items que cada usuario tiene item_id <br>

user_id_counts = df_user_id_genres.groupby('user_id')['item_id'].count().reset_index(name='count')

> Filtramos el dataframe para que incluya solo los usuarios e informacion de aquellos que tienen mas de 200 juegos comprados( definimos un threshold de  200), asi reducimos el peso del df para la funcion, sino no correrá en render ya que al descomprimir el .parquet se exceden los 500MB disponibles en render.<br>

filtered_user_ids = user_id_counts[user_id_counts['count'] > 200]['user_id']

>Filtramos el df_user_id_genres basados en user_ids<br>

filtered_df = df_user_id_genres[df_user_id_genres['user_id'].isin(filtered_user_ids)]
filtered_df

De esta manera, se redujeron a 2MM de registros, pero manteniendo informacion valiosa para poder desplegar la función sin problemas de capacidad, y el file comprimido pesó menos de 9MB<br>

EL detalle de todo lo trabajado se encuentra en [Feature Engineering](03.%20Feature%20Engineering.ipynb)  donde encuentra el paso a paso completo para crear cada una de estas funciones y sus marcos de datos de entrada, métodos de filtrados interactivos y todo lo relativo al desarrollo y creación de las mismas.

[Deploy an API in Render](https://medium.com/@iamgreatdiro/deploying-apis-on-render-a-step-by-step-guide-4ebe6a3fd377)

# 04. Modelo de Recomendación
En este apartado van a encontrar el proceso paso a paso para construir nuestros dos modelos de recomendación:
<br>
1. **Item - Item** utilizando Filtrado Colaborativo.
2. **User - item** para esto creamos nuestro modelo utilizando Similitud Coseno y Modelo de Aprendizaje Automático KNN.
<br>
Las funciones que creamos son las solicitadas:<br>

>A. **Find Similar Games to the one we input**: Ingresamos un juego y, basándonos en las críticas y calificaciones de los usuarios, encuentra una lista de usuarios con gustos similares (usuarios similares que calificaron con 5 el juego ingresado) y, en función de estos usuarios, calificamos los juegos que a estos usuarios les gustaron y proporcionamos una lista de 5 juegos para recomendar.<br>
>B. **Find Similar Games to the User we input**: Ingresamos un user_id y, en función de las similitudes del usuario con otros usuarios y calificaciones, utilizando filtrado colaborativo proporcionamos una lista de 5 juegos similares a los juegos que al usuario ingresado le podrían gustar.
<br>

# 1. Filtrado Colaborativo
El filtrado colaborativo es una técnica popular utilizada en sistemas de recomendación para hacer predicciones automáticas (filtrado) sobre los intereses de un usuario al recopilar información de preferencias o gustos de muchos usuarios (colaboración). La idea básica detrás del filtrado colaborativo es recomendar elementos a un usuario en función de las preferencias de otros usuarios que tienen gustos o preferencias similares.

Hay dos tipos principales de filtrado colaborativo:

- Filtrado colaborativo basado en **User-based**: En este enfoque, el sistema recomienda elementos a un usuario en función de las preferencias de otros usuarios que son similares a ese usuario. El sistema identifica usuarios similares al comparar sus calificaciones o comportamientos en elementos.

- Filtrado colaborativo basado en **Item-based**: En este enfoque, el sistema recomienda elementos a un usuario en función de las similitudes entre elementos. El sistema identifica elementos similares mediante el análisis de las calificaciones o comportamientos de los usuarios en esos elementos.

Tanto el filtrado colaborativo basado en usuarios como en elementos dependen de la noción de similitud entre usuarios o elementos. Métricas de similitud como la similitud de coseno, la correlación de Pearson o la similitud de Jaccard a menudo se utilizan para medir la similitud entre usuarios o elementos.
En nuestro proyecto, utilizamos la similitud de coseno por su simplicidad y buen rendimiento.

**El filtrado colaborativo no requiere ninguna información sobre los elementos o usuarios en sí; en cambio, se basa únicamente en las interacciones pasadas o calificaciones de los usuario**

### Paso 1- Creamos un motor de búsqueda<br>

### 1.1 Función para limpiar item_name (Títulos de juegos) con REGEX
Para esto, crearemos una función para limpiar el título de nuestro conjunto de datos. Esto significa que la columna de título de nuestro conjunto de datos será la entrada y la salida será una cadena limpia de caracteres especiales. Esto será la entrada de nuestro motor de búsqueda.

### 1.2 Creando una Matriz TFIDF
Utilizando la libreria sklearn, en particular el metodo TfidfVectorizer. Se crea el motor de busqueda para que ingresando un titulo de un juego, nos devuelva 5 titulos similares al que ingresamos.
En uno de los tutoriales que investigué, utilizabas Jupyter Widgets para visualizr un text box y la devolución de la función. Fue algo muy entretenido y se crea una interfaz del usuario muy atractiva con solo unas lineas de codigo<br>
Corriendo el codigo del Notebook se puede apreciar la misma y como funciona el motor de búsqueda.<br>

### 1.3 Transformación de Revisiones
Necesitamos transformar las revisiones de nuestros usuarios, el resultado del botón de recomendación y la calificación de útil (donde otros usuarios calificaron la revisión) en una calificación para alimentar nuestro sistema de recomendación.<br>
Para esto creamos una función que considerará los tres elementos, cuando existan y promediará una calificación que exprese la revisión del usuario en números.<br>
En general, esta función asigna una puntuación de calificación en función de los valores proporcionados para 'rate', 'sentiment' y 'recommend', considerando diferentes combinaciones de estos valores y siguiendo reglas específicas.<br>

(Esto ya fue comentado en el apartado 3.2 de Featuring Engineering, revisar nuevamente por si no fuera claro o se quisiera mayor detalle)

Adicionalmente, para poder utilizar esta valiosa información por el modelo de Recomendación procedo a normalizar la columna en un ranking de 1 a 5, siendo 5 la mejor calificacion (recomendado y sentiment==2 y rating 100) y 1 la menor calificación (sentiment == 0, recomender == False y rating == 0)

### 1.4 Recomendar una lista de 5 juegos basada en un juego
Con esta tarea completa, me propuse crear una función que, aplicando filtrado interactivo y colaborativo, proporcionará una lista de 5 juegos basada en una entrada siendo el input el nombre de un juego.
En esta primera función, la entrada es el item_id, que es el identificador del juego.
El filtrado colaborativo son filtros interactivos 
#### 1.4.1 Encontrar usuarios que les gustó el mismo juego
En resumen, la función creada, find_similar_games, implementa un enfoque de filtrado colaborativo para recomendar juegos similares a uno dado (game_id). Funciona de la siguiente manera:
- Primero, identifica los usuarios que han calificado el juego dado con una calificación de 5 o superior.
- Luego, encuentra otros juegos que hayan sido calificados con una alta puntuación por estos mismos usuarios.
- Calcula la proporción de usuarios similares que han calificado cada uno de estos juegos, así como la proporción de todos los usuarios que han calificado estos juegos.
- Finalmente, calcula un puntaje para cada juego basado en estas proporciones y devuelve los 5 juegos con los puntajes más altos, junto con sus nombres y géneros.
**Este enfoque permite recomendar juegos que son populares entre los mismos usuarios que disfrutaron del juego dado, lo que tiende a producir recomendaciones más personalizadas y relevantes.**

#### 1.4.2 Find_Similar_Games(user_id)<br>
Esta función, find_similar_games, toma como entrada un user_id y devuelve una lista de 5 juegos recomendados para ese usuario. Utiliza un enfoque de filtrado colaborativo para encontrar juegos que son populares entre usuarios similares al usuario dado. El proceso se puede resumir de la siguiente manera:
- Identifica otros usuarios que tienen gustos similares al usuario dado.
- Encuentra juegos que son populares entre estos usuarios similares.
- Calcula un puntaje para cada juego basado en su popularidad entre los usuarios similares.
- Devuelve los 5 juegos con los puntajes más altos como recomendaciones para el usuario dado.
**Este enfoque permite recomendar juegos que son populares entre usuarios con gustos similares, lo que puede resultar en recomendaciones más relevantes y personalizadas para el usuario dado.**

El desarrollo en detalle del modelo se encuentra en [Recomender System](04.%20Recomendation_Model.ipynb), por favor ir a este notebook para ver mas en detalle el paso a paso del mismo.<br>

## **Detección de Oportunidades**
Como el motor de búsqueda se basa en el df_games que tiene mas de 58K registros de la plataforma, juegos, pero la recomendación se hace en base a las reseñas de los usuarios, que solo cuenta con recomendaciones sobre 3.682 juegos, muchas veces no produce recomendaciones. Esto se debe a que faltaría completar o expandir la base de reseñas para así abarcar mayor cantidad de juegos.<br>
Cuando no produzca una recomendación no es por una falla de la función sino mas bien por una ineficiencia de los datos originales<br>
Esto podría mejorarse fácilmente y haría más asertivo al modelo, ya que se podría incrementar el threshold de similitudes al 10% (actualmente corre al 50%), entre otras posibilidades.
Al estar el modelo construído, solo hay que alimentarle bases de datos más amplias y mejoraría rápidamente la performance<br>

# 2. Recomendador Usuario - item con Aprendizaje Automático KNN 
Acá construímos otro modelo de Recomendación pero utilizando el algoritmo de Vecinos más Cercanos (KNN) con similitud de coseno para sugerir juegos similares basados en las preferencias del usuario. El modelo opera mediante la creación de una matriz dispersa que representa las interacciones entre usuarios y juegos. Luego, calcula las similitudes entre juegos utilizando la distancia coseno e identifica los vecinos más cercanos para cada juego. Se utiliza la coincidencia difusa de cadenas para mejorar la coincidencia de nombres de juegos. Finalmente, el modelo devuelve una lista de juegos recomendados basados en el nombre del juego proporcionado como entrada.

### 2.1 Preparación de Datos
Preparamos la informacion a utilizar para el desarrollo del modelo creando a partir de dataframe Reviews, una matriz csr. Para ello utilizamos la libreria scipy.<br>

### 2.2 Definimos el modelo y sus métricas a partir de esta matriz
Asignar el coseno como métrica en el modelo KNN significa que la búsqueda encuentra elementos de la matriz de juegos-usuarios con una similitud en cada calificación. Por lo tanto, ahora, dado un usuario o un elemento, buscará similitudes en la matriz de juegos y usuarios.<br>
Los parametros elegidos son básicos, ee establece en "brute" para que la búsqueda itere en toda la matriz.<br>

### 2.3 Creamos nuevamente dos funciones al igual que con el modelo anterior:

#### 2.3.1 Encontrar usuarios que les gustó el mismo juego
Este modelo de recomendación utiliza el algoritmo de Vecinos más Cercanos (KNN) con similitud de coseno para sugerir juegos similares basados en las preferencias del usuario. El modelo opera mediante la creación de una matriz dispersa que representa las interacciones entre usuarios y juegos. Luego, calcula las similitudes entre juegos utilizando la distancia coseno e identifica los vecinos más cercanos para cada juego. Se utiliza la coincidencia difusa de cadenas para mejorar la coincidencia de nombres de juegos. Finalmente, el modelo devuelve una lista de juegos recomendados basados en el nombre del juego proporcionado como entrada.

#### 2.3.2  User_recommender
Esta función, llamada "user_recommender", recomienda juegos para un usuario específico basándose en los juegos que el usuario ya ha jugado. Primero, verifica si el usuario está presente en el conjunto de datos de interacciones entre usuarios y juegos. Luego, encuentra los juegos que el usuario aún no ha jugado. Utiliza el algoritmo de los k vecinos más cercanos (KNN) para encontrar juegos similares a los que el usuario ha jugado. Finalmente, devuelve una lista de juegos recomendados para el usuario.

### Guardamos los Dataframe 
Ahora podemos guardar los  dataframes ya que estos serán los utilizados por las funcinoes que desarrollaremos en el 3.2 del  [Featuring Engineering](03.%20Feature%20Engineering.ipynb).
Elegí guardar los mismos en formato .parquet para optimizar el espacio <br>


